# Quickstart

The latest release of TAPE is installable via pip, using the following command:

```
pip install lf-tape
```

For more detailed installation instructions, see the [Installation Guide](installation.html).

TAPE provides a scalable framework for analyzing astronomical time series data. Let's walk through a brief example where we calculate the Structure Function for a set of spectroscopically confirmed QSOs. First, we grab the available TAPE Stripe 82 QSO dataset:

In [ ]:
from tape import Ensemble

ens = Ensemble()  # Initialize a TAPE Ensemble
ens.from_dataset("s82_qso")

This dataset contains 9,258 QSOs, we can view the first 5 entries in the "object" table to get a sense of the available object-level information:

In [ ]:
ens.head("object", 5)

The Ensemble stores data in two `dask` dataframes, object-level information in the "object" table as shown above, and individual time series measurements in the "source" table. As a result, many operations on the Ensemble closely follow operations on `dask` (and by extension `pandas`) dataframes. Let's filter down our large QSO set to a smaller set with the total number of observations per object within a certain range:

In [ ]:
ens.calc_nobs()  # calculates number of observations, produces "nobs_total" column
ens = ens.query("nobs_total >= 95 & nobs_total <= 105", "object")

We can now view the entirety of our remaining QSO set:

In [ ]:
ens.compute("object")

Finally, we can calculate the Structure Function for each of these QSOs, using the available TAPE Structure Function Module:

In [ ]:
from tape.analysis import calc_sf2

result = ens.batch(
    calc_sf2, sf_method="macleod_2012"
)  # The batch function applies the provided function to all individual lightcurves within the Ensemble
result.compute()

The result is a table of delta times (dts) and structure function (sf2) for each unique lightcurve (labeled by lc_id).